# Group No: 18

## Group Members Names:

 1. PEYALA SAMARASIMHA REDDY - 2023AA05072
 2. PEGALLAPATI SAI MAHARSHI - 2023AA05924
 3. ANIRUDDHA DILIP MADURWAR - 2023AA05982
 4. TUSHAR DEEP - 2023AA05885

## Assignment - 2

Real-Time Prediction with Apache Kafka and Neural Networks

## Producer File

- Here, we have setup the Apache Kafka locally, we started the Kafka and Zookeeper. Then we will run this producer and consumer via jupyterbooks. 
- This producer.ipynb loads the csv file for test data and streams, send to the consumer for predictions.
- For simplicity we only sending the last 100 rows of the dataset.

In [12]:
import pandas as pd
import json
import time
import logging
from kafka import KafkaProducer

# Define log file name
log_file = "producer_logs.log"

# Setup Logging: Remove existing handlers and reconfigure
logging.getLogger().handlers.clear()
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"  # Use "w" to overwrite or "a" to append logs
)

# Create a console handler to also print logs to the console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
console_handler.setFormatter(console_formatter)

# Add the console handler to the logger
logger = logging.getLogger()
logger.addHandler(console_handler)

# Loading the dataset
csv_file = "california_housing.csv"
logging.info(f"Loading data from {csv_file}...")

df = pd.read_csv(csv_file)

logging.info("Data loaded successfully!")

# Read last 100 rows of the dataset as test set
logging.info("Reading last 100 rows of the dataset as test set...")
df = df.iloc[-100:].reset_index(drop=True)

# Define feature and target columns
feature_columns = df.columns[:-1]  # All columns except the last one
target_column = df.columns[-1]  # The last column is the target

# Kafka Producer Setup
producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

logging.info("\nStarting Data Streaming to Kafka...\n")

# Track total messages sent
total_messages_sent = 0

# Send data row by row
for i, row in df.iterrows():
    message = {
        "id": i,
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "features": row[feature_columns].tolist(),
        "actual_price": row[target_column]
    }

    # Send message to Kafka topic
    producer.send("input-data", value=message)
    
    # Log the message
    log_message = f"Sent Message ID: {i}, Timestamp: {message['timestamp']}, Features: {row[feature_columns].tolist()}, Actual Price: {row[target_column]:.4f}"
    logging.info(log_message)
    
    # Print message details
    print(f"Sent Message {i}:")
    for feature_name, feature_value in zip(feature_columns, row[feature_columns]):
        print(f"   - {feature_name}: {feature_value:.4f}")
    print(f"   - Actual Price (Target): {row[target_column]:.4f}\n")

    total_messages_sent += 1  # Count messages sent
    time.sleep(1)  # Simulate real-time streaming

# Ensure all messages are sent before closing
producer.flush()
producer.close()

# Final summary message
summary_message = f"\nData Streaming Completed: {total_messages_sent} messages sent to Kafka!"
logging.info(summary_message)
print(summary_message)

# Confirmation message about log file
log_saved_message = f"All logs are saved in: {log_file}"
logging.info(log_saved_message)
print(log_saved_message)


2025-03-16 18:07:01,604 - INFO - Loading data from california_housing.csv...
2025-03-16 18:07:01,618 - INFO - Data loaded successfully!
2025-03-16 18:07:01,619 - INFO - Reading last 100 rows of the dataset as test set...
2025-03-16 18:07:01,620 - INFO - <BrokerConnection client_id=kafka-python-producer-12, node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
2025-03-16 18:07:01,621 - INFO - Broker version identified as 2.6
2025-03-16 18:07:01,622 - INFO - <BrokerConnection client_id=kafka-python-producer-12, node_id=bootstrap-0 host=localhost:9092 <checking_api_versions_recv> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.
2025-03-16 18:07:01,623 - INFO - 
Starting Data Streaming to Kafka...

2025-03-16 18:07:01,625 - INFO - Sent Message ID: 0, Timestamp: 2025-03-16 18:07:01, Features: [3.1908, 16.0, 4.386792452830188, 0.981132075471698, 1386.0, 2.179245283018868, 38.54, -121.72], Actual Price: 1.943

Sent Message 0:
   - MedInc: 3.1908
   - HouseAge: 16.0000
   - AveRooms: 4.3868
   - AveBedrms: 0.9811
   - Population: 1386.0000
   - AveOccup: 2.1792
   - Latitude: 38.5400
   - Longitude: -121.7200
   - Actual Price (Target): 1.9430



2025-03-16 18:07:02,627 - INFO - Sent Message ID: 1, Timestamp: 2025-03-16 18:07:02, Features: [4.6225, 13.0, 6.115695067264574, 1.0385650224215246, 2828.0, 2.536322869955157, 38.54, -121.7], Actual Price: 2.2650


Sent Message 1:
   - MedInc: 4.6225
   - HouseAge: 13.0000
   - AveRooms: 6.1157
   - AveBedrms: 1.0386
   - Population: 2828.0000
   - AveOccup: 2.5363
   - Latitude: 38.5400
   - Longitude: -121.7000
   - Actual Price (Target): 2.2650



2025-03-16 18:07:03,632 - INFO - Sent Message ID: 2, Timestamp: 2025-03-16 18:07:03, Features: [4.7308, 33.0, 6.575596816976128, 1.013262599469496, 979.0, 2.5968169761273208, 38.55, -121.75], Actual Price: 2.3620


...

Sent Message 98:
   - MedInc: 1.8672
   - HouseAge: 18.0000
   - AveRooms: 5.3295
   - AveBedrms: 1.1719
   - Population: 741.0000
   - AveOccup: 2.1232
   - Latitude: 39.4300
   - Longitude: -121.3200
   - Actual Price (Target): 0.8470



2025-03-16 18:08:41,100 - INFO - Sent Message ID: 99, Timestamp: 2025-03-16 18:08:41, Features: [2.3886, 16.0, 5.254716981132075, 1.1622641509433962, 1387.0, 2.616981132075472, 39.37, -121.24], Actual Price: 0.8940


Sent Message 99:
   - MedInc: 2.3886
   - HouseAge: 16.0000
   - AveRooms: 5.2547
   - AveBedrms: 1.1623
   - Population: 1387.0000
   - AveOccup: 2.6170
   - Latitude: 39.3700
   - Longitude: -121.2400
   - Actual Price (Target): 0.8940



2025-03-16 18:08:42,104 - INFO - <BrokerConnection client_id=kafka-python-producer-12, node_id=0 host=hitloop:9092 <connected> [IPv4 ('127.0.1.1', 9092)]>: Closing connection. 
2025-03-16 18:08:42,107 - INFO - 
Data Streaming Completed: 100 messages sent to Kafka!
2025-03-16 18:08:42,108 - INFO - All logs are saved in: producer_logs.log



Data Streaming Completed: 100 messages sent to Kafka!
All logs are saved in: producer_logs.log
